In [17]:
import re
import os
import json
import numpy as np
from tqdm import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# Create the elastic instance
elastic = Elasticsearch(
    "http://localhost:9200",
    request_timeout=1000000
)

# Successful response!
elastic.info()

ObjectApiResponse({'name': 'ubuntu', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'nnVPfeLWRgunp94gp_U7Eg', 'version': {'number': '8.3.2', 'build_type': 'tar', 'build_hash': '8b0b1f23fbebecc3c88e4464319dea8989f374fd', 'build_date': '2022-07-06T15:15:15.901688194Z', 'build_snapshot': False, 'lucene_version': '9.2.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [2]:
a = torch.load(r"C:\Users\Pt\Downloads\nq_model.ckpt", map_location="cpu")

ModuleNotFoundError: No module named 'pytorch_lightning'

In [15]:
elastic.search(
    index="wenshu",
    size=2,
    from_=0,
    _source=False,
    fields=["case_name", "content", {"field": "publish_date", "format": "year_month_day"}, "court_name"],
    query={
        "combined_fields": {
            "query": "测试",
            "fields": ["case_name", "content"]
        }
    },
    aggs={
        "agg-court": {
            "terms": {
                "field": "court_name"
            }
        }
    },
    # highlight={
    #     "fields": {
    #         "content": {
    #             "pre_tags" : ["<strong>"],
    #             "post_tags": ["</strong>"],
    #             "number_of_fragments": 1,
    #         }
    #     }
    # },
    # post_filter={
    #     "bool": {
    #         "filter": [
    #             {
    #                 "terms": {"court_name": ["德阳市中级人民法院"]}
    #             },
    #             {
    #                 "terms": {"case_type": ["管辖案件"]}
    #             }
    #         ]
    #     }
    # }
)["aggregations"]

{'agg-court': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 61,
  'buckets': [{'key': '金华市婺城区人民法院', 'doc_count': 3},
   {'key': '上海市嘉定区人民法院', 'doc_count': 2},
   {'key': '太原市万柏林区人民法院', 'doc_count': 2},
   {'key': '宁明县人民法院', 'doc_count': 2},
   {'key': '新昌县人民法院', 'doc_count': 2},
   {'key': '湖南省长沙市雨花区人民法院', 'doc_count': 2},
   {'key': '东莞市第二人民法院', 'doc_count': 1},
   {'key': '临武县人民法院', 'doc_count': 1},
   {'key': '义乌市人民法院', 'doc_count': 1},
   {'key': '云南省昆明市官渡区人民法院', 'doc_count': 1}]}}

In [ ]:
emb = np.random.rand(768).astype(np.float32)
elastic.knn_search(
    index="wenshu", 
    knn={
        "field": "vector",
        "query_vector": (emb / np.linalg.norm(emb)).tolist(),
        "k": 10,
        "num_candidates": 10
    },
    source=False,
    fields=["case_name", "content"]
)

In [34]:
file = "p2-1.filtered"
embeddings = np.memmap(
    os.path.join("data/encode", "DPR", "wenshu", file, "text_embeddings.mmp"),
    dtype=np.float32,
    mode="r"
).reshape(-1, 768)

def gendata():
    with open(f"../../../Data/wenshu/{file}.txt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            case = json.loads(line.strip())
            del case["crawl_time"]
            del case["legal_base"]
            del case["tf_content"]
            del case["id"]
            case["vector"] = embeddings[i].tolist()
            yield case

for x in tqdm(gendata(), desc="Indexing", total=embeddings.shape[0]):
    elastic.index(index="wenshu", document=x)

Indexing:  40%|███▉      | 201062/505530 [37:19<56:31, 89.77it/s]   


KeyboardInterrupt: 

In [32]:
with open("/home/peitian_zhang/Data/wenshu/p2-1.txt") as f:
    for i,x in enumerate(f):
        if i == 0:
            print(x[:50])
        if i == 100001:
            print(x[:50])
            break

{"id": "4", "doc_id": "dd582a41-065d-44af-b86a-c5e
{"id": "1784033", "doc_id": "424e5b83-2c86-49d5-9d


In [33]:
with open("/home/peitian_zhang/Data/wenshu/p2-1.filtered.txt") as f:
    for i,x in enumerate(f):
        if i == 0:
            print(x[:50])

{"id": "1784033", "doc_id": "424e5b83-2c86-49d5-9d


In [9]:
elastic.indices.create(
    index="wenshu",
    settings={
        'analysis': {
            'analyzer': {
                # we must set the default analyzer
                "default": {
                    "type": "smartcn"
                }
            }
        },
        "index.mapping.ignore_malformed": True
    },
    mappings={
        "properties": {
            # field name
            "doc_id": {
                "type": "keyword",
            },
            "court_name": {
                "type": "keyword",
            },
            "court_id": {
                "type": "keyword",
            },
            "court_province": {
                "type": "keyword",
            },
            "court_city": {
                "type": "keyword",
            },
            "court_region": {
                "type": "keyword",
            },
            "court_district": {
                "type": "keyword"
            },
            "pub_prosecution_org": {
                "type": "keyword"
            },
            "case_type": {
                "type": "keyword",
            },
            "cause": {
                "type": "keyword",
            },
            "trial_round": {
                "type": "keyword"
            },
            "content": {
                "type": "text"
            },
            "vector": {
                "type": "dense_vector",
                "dims": 768,
                # enable hnsw
                "index": True,
                # inner product only allows unit-length vector
                "similarity": "dot_product"  
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'wenshu'})

In [ ]:
# test chinese index
elastic.indices.analyze(index="test-chinese", text="中国人民大学")

In [8]:
# delete index
elastic.indices.delete(index="wenshu")

ObjectApiResponse({'acknowledged': True})

In [ ]:
elastic.indices.get_alias(index="*")

In [ ]:
elastic.search(
    index="test-wenshu",
    query={
        "match_all": {}
    }
)["hits"]["hits"]

In [ ]:
elastic.delete_by_query(
    index="wenshu", 
    query={
        "match_all": {}
    }
)

In [27]:
# filter text
skipped = 0
file = "p2-1"
with open(f"../../../Data/wenshu/{file}.txt", encoding="utf-8") as f, open(f"../../../Data/wenshu/{file}.filtered.txt", "w", encoding="utf-8") as g:
    for i, line in enumerate(tqdm(f)):
        if i <= 100000:
            continue

        fields = json.loads(line.strip())

        if fields["basics_text"]:
            content = fields["basics_text"].replace("/n", "")
        else:
            content = fields["content"]
            if not content:
                skipped += 1
                continue
            else:
                content = content.replace("\n", "")
                for x in ("指控：", "经审理查明，", "诉称：", "诉讼请求：", "理由："):
                    idx = content.rfind(x)
                    if idx != -1:
                        break
                if idx == -1:
                    skipped += 1
                    # print(content)
                    # if skipped > 5:
                    #     break
                    continue
                else:
                    content = content[idx:]
        # write filter content to a new file
        fields["tf_content"] = content
        g.write(json.dumps(fields, ensure_ascii=False) + "\n")

        if i > 1000000:
            break
skipped

1000002it [01:50, 9034.40it/s]


394472

In [21]:
elastic.search(
    index="wenshu",
    query={
        "match_all": {
        }
    }
)["hits"]["total"]

{'value': 10000, 'relation': 'gte'}

In [22]:
elastic.count(
    index="wenshu",
    query={
        "match_all": {
        }
    }
)

ObjectApiResponse({'count': 53794, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})